## Harris Corner Detection
When detecting image features, corners are very important. Just like
what humans can do, we can easily localize a corner from an image, but
much harder based on an edge and even impossible from a plain. And also,
blobs(some small and strange shapes) can also make us easy to find.<br>
So we first need to find corners, and that is what Harris Corner Detection
does.<br>
Harris Corner Detection uses a simple method based on the intensity variation
in images.
- First, we need to calculate the intensity difference<br>
  $$E(u, v) = \sum_{x, y} w(x, y)[I(x+u, y+v) - I(x, y)]^2$$<br>
  The $w(x, y)$ is the weight in each pixel, we can use mean weight or Gaussian
  weight and so on. The $I(x,y)$ is the intensity.
- Second, we just use Taylor Expansion to approximate the equation<br>
  $$E(u, v) \approx [u, v]M[u, v]^{T}$$<br>
  $$
    M = \sum_{x,y}w(x, y)\begin{bmatrix}
    I_xI_x & I_xI_y \\
    I_xI_y & I_yI_y
    \end{bmatrix}
  $$<br>
  $I_x$ is the derivative of intensity in x direction. And
  $I_y$ is the derivative of intensity in y direction.
- Third, we use a score to judge if it is a corner.
  Because we want to make the $E$ as big as possible.
  If we calculate the derivative of $E$, we will find
  that it will just depends on $M$. So we can just
  calculate the score base on $M$<br>
  $$
    R = det(M) - k (trace(M))^2
  $$<br>
  $det(M) = \lambda_1 * \lambda_2$<br>
  $trace(M) = \lambda_1 + \lambda_2$<br>
  $\lambda_1$ and $\lambda_2$ are the 2 eigenvalues of $M$<br>

Concretely, the $det$ and $trace$ rely on the variation of intensity.
When $\lambda$ is big, the intensity variation is acute, so it is
possible to be a corner here. And if small, it is probablity that
there is a plain.
The evaluation rules are as below:
1. When $|R|$ is small, which happens when $\lambda_1$ and $\lambda_2$
are small, the region is flat.
2. When $R<0$, which happens when $\lambda_1 >> \lambda_2$ or
vice versa, the region is edge.
3. When $R$ is large, which happens when $\lambda_1$ and $\lambda_2$ are
large and $\lambda_1 \sim \lambda_2$, the region is a corner.

In [1]:
import numpy as np
import cv2 as cv

filename = '../img/chessboard2.jpeg'
img = cv.imread(filename)
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

gray = np.float32(gray)
# blockSize means the window size, in which neighbours should be calculate the intensity variation
# ksize means the Sobel operator size
# k is the free parameter we described in the equation above
dst = cv.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)

# dilate the corner
dst = cv.dilate(dst, None)
img[dst>0.01*dst.max()]=[0, 0, 255]
cv.imshow('dst', img)
cv.waitKey(0)
cv.destroyAllWindows()